In [3]:
from pulp import *

In [11]:
def minMax(Tab):
    Min = Tab[0][0]
    Max = Tab[0][0]
    for liste in Tab:
        for elem in liste:
            if elem >Max :
                Max = elem
            elif elem < Min:
                Min = elem 
    return (Min,Max)

def LinProg(Tab):

    # Variables
    variables = []
    for i in range(len(Tab)):
        variables.append(LpVariable("p"+str(i), 0, 1))
    weights = Tab

    # Solving the problem from the vectors Variables and weights
    prob = LpProblem("test1", LpMaximize)

    (Min,Max) = minMax(weights)
    t = LpVariable("t",Min,Max)
    # Objective
    prob += t

    # Constraints
    for column in range(len(weights)):
        utility = 0
        for row in range(len(weights[0])):
            utility += variables[row]*weights[column][row]
        prob += utility - t >= 0
    # prob += -p1 - t >= 0
    # prob += p1-p2 - t >= 0
    probabilities = 0
    for variable in variables:
        probabilities += variable
    prob += probabilities == 1

    GLPK().solve(prob)

    P = []
    # Solution
    for v in prob.variables():
        #print(v.name, "=", v.varValue)
        if v.name != "t":
            P.append(v.varValue)

    #print("objective=", value(prob.objective))

    return(P, value(prob.objective))

In [12]:
def len1(x):
    l = -1
    for i in range(len(x)):
        if x[i] == 1 and l > -1:
            return -1
    
        elif x[i] == 1:
            l = i
            
    return l

In [13]:
def G(x,y,t):
    
    # First the initialization cases
    if t == 0 and x == y:
        return 1
    elif x == 0: 
        if t < 0: 
            return 0
        elif y == t:
            return 1
        elif y < t:
            return 3
        elif y > t:
            return 0
        else:
            raise ValueError("Impossible case")
    elif y == 0: 
        if t > 0:
            return 3
        elif (x+t) == 0:
            return 1
        elif x > t:
            return 3
        elif x < t:
            return 0
        else:
            raise ValueError("Impossible case")
    else:
        # Here is the reccurence
        
        # We enumerate all the reachable states
        states = enumStatesfrom(x,y,t)
        
                
        #If there is only one issue for the player 1, the profits will be the argmin of all the issues from the player 2
        if x == 1: 
            x,y,t = states[0][0]
            mini = G(x,y,t)
            for state in states[0]:
                x,y,t = state
                profits = G(x,y,t)
                if  profits < mini:
                    mini = profits
            return mini
        
        # If there is more the one issue for the player 1 we try to reduce the number of issues by finding dominated ones
        p = []
        for x1 in range(len(states)):
            p.append(0)
            for y in range(len(states[0])):
                for x2 in range(len(states)):
                    if x1 < x2 :
                        p[x1] = 1        

        
        if len1(p) > -1: 
            x,y,t = states[len1(p)][0]
            mini = G(x,y,t)
            for state in states[len1(p)]:
                x,y,t = state
                profits = G(x,y,t)
                if  profits < mini:
                    mini = profits
            return mini
        
    
                        
        # Now we transform the game into Linear Programming problem
        
        
        
        # Example of Linear Programming solving using pulp
#        prob = LpProblem("test1", LpMinimize) 
#        # Variables 
#        x = LpVariable("x", 0, 4) 
#        y = LpVariable("y", -1, 1) 
#        z = LpVariable("z", 0) 

#        # Objective 
#        prob += x + 4*y + 9*z 

#        # Constraints 
#        prob += x+y <= 5 
#        prob += x+z >= 10 
#        prob += -y+z == 7 

#        GLPK().solve(prob) 

#        # Solution 
#        for v in prob.variables(): 
#            print(v.name, "=", v.varValue)

#        print("objective=", value(prob.objective))
        
        
        
        raise NotImplementedError("Missing cases on the reccurence")

In [14]:
def enumStatesfrom(n1,n2,t):
    Liste = []
    for x in range(1, n1+1):
        L = []
        for y in range(1, n2+1):
            if x == y :
                L.append((n1-x, n2-y, t))
            elif x > y :
                L.append((n1-x, n2-y, t+1))
            elif x < y :
                L.append((n1-x, n2-y, t-1))
            else :
                raise ValueError("Impossible case")
        Liste.append(L)
    return Liste

In [15]:
print(enumStatesfrom(2,3,1))

[[(1, 2, 1), (1, 1, 0), (1, 0, 0)], [(0, 2, 2), (0, 1, 1), (0, 0, 0)]]


In [9]:
print(G(1,2,1))

1


In [10]:
print(G(2,3,1))

1
